In [1]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers trl peft accelerate bitsandbytes

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",      # New Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",           # Llama-3 15 trillion tokens model 2x faster!
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",        # Phi-3 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",             # Gemma 2.2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

/home/paulaperezbianchi/Documents/nlp-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2024-06-29 14:28:36.155064: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-29 14:28:36.180386: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-29 14:28:36.586199: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


==((====))==  Unsloth: Fast Llama patching release 2024.6
   \\   /|    GPU: NVIDIA GeForce RTX 3060. Max memory: 11.754 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 8.6. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj","gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.6 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


<a name="Data"></a>
### Data Prep
We now use the Alpaca dataset from [yahma](https://huggingface.co/datasets/yahma/alpaca-cleaned), which is a filtered version of 52K of the original [Alpaca dataset](https://crfm.stanford.edu/2023/03/13/alpaca.html). You can replace this code section with your own data prep.

**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

**[NOTE]** Remember to add the **EOS_TOKEN** to the tokenized output!! Otherwise you'll get infinite generations!

If you want to use the `llama-3` template for ShareGPT datasets, try our conversational [notebook](https://colab.research.google.com/drive/1XamvWYinY6FOSX9GLvnqSjjsNflxdhNc?usp=sharing).

For text completions like novel writing, try this [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing).

In [3]:
prompt = """ 
Below you will find the title, summary, and body of a news article. 
Your task is to analyze these components and classify whether the article is sensationalist or not.

Sensationalist is defined as: "presenting information in a way that is intended to provoke public interest, excitement, or anxiety, at the expense of accuracy."

### Article information:
    Title: {}
    Subheading: {}
    Body: {}
    is this article false?: {}
    
### is this article sensationalist?:
{}
"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    
    title = examples["Titular"]
    summary = examples["Copete"]
    body      = examples["Cuerpo"]
    is_sensationalist      = examples["Amarillismo"]
    is_false = examples["Falsa"]
    texts = []
    for title, summary, body, is_sensationalist, is_false in zip(title, summary, body, is_sensationalist, is_false):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = prompt.format(title, summary, body, is_sensationalist, is_false) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

In [16]:
# format csv
import pandas as pd 
sensationalist_dataset1 = pd.read_csv('data/trainset_realnews_and_corpus.csv', encoding='latin1', index_col='Id')
sensationalist_dataset2 = pd.read_csv('data/trainset_synthetic_data.csv', encoding='latin1', index_col='Id')

#Training dataset 
sensationalist_dataset = pd.concat([sensationalist_dataset1, sensationalist_dataset2], ignore_index=True)
sensationalist_dataset.to_json('data/trainset.json', orient='index')

In [19]:
def load_dataset(dataset_path):
    dataset = pd.read_json(dataset_path, orient='index',  encoding='latin1')
    return dataset

In [10]:
from datasets import load_dataset, Dataset

#dataset = load_dataset("json", data_files= "data/dataset_gpt_utf8.json", split='train')
#dataset = dataset.map(formatting_prompts_func, batched = True,)

dataset_dir = formatting_prompts_func(sensationalist_dataset)
dataset = Dataset.from_dict(dataset_dir)


<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [13]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        max_steps = 1000,
        warmup_steps = 5,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2): 100%|██████████| 99/99 [00:00<00:00, 226.25 examples/s]
max_steps is given, it will override any value given in num_train_epochs


In [14]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 3060. Max memory = 11.754 GB.
5.594 GB of memory reserved.


In [15]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 99 | Num Epochs = 84
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 1,000
 "-____-"     Number of trainable parameters = 41,943,040
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: paulitapb (paps). Use `wandb login --relogin` to force relogin


  0%|          | 1/1000 [00:06<1:44:09,  6.26s/it]

{'loss': 2.1963, 'grad_norm': 0.9739549160003662, 'learning_rate': 4e-05, 'epoch': 0.08}


  0%|          | 2/1000 [00:11<1:34:01,  5.65s/it]

{'loss': 2.2009, 'grad_norm': 1.005650281906128, 'learning_rate': 8e-05, 'epoch': 0.16}


...

{'loss': 0.0267, 'grad_norm': 0.08575721085071564, 'learning_rate': 2.0100502512562817e-07, 'epoch': 79.92}


100%|██████████| 1000/1000 [1:20:28<00:00,  4.69s/it]

{'loss': 0.026, 'grad_norm': 0.09506573528051376, 'learning_rate': 0.0, 'epoch': 80.0}


100%|██████████| 1000/1000 [1:20:29<00:00,  4.83s/it]

{'train_runtime': 4830.9779, 'train_samples_per_second': 1.656, 'train_steps_per_second': 0.207, 'train_loss': 0.06467841903679072, 'epoch': 80.0}


In [17]:
# Save the LoRA weights
model.save_pretrained("model_weights")

In [20]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

197.8541 seconds used for training.
3.3 minutes used for training.
Peak reserved memory = 7.471 GB.
Peak reserved memory for training = 1.242 GB.
Peak reserved memory % of max memory = 63.561 %.
Peak reserved memory for training % of max memory = 10.567 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

In [19]:
def inference(article_data):
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference
    inputs = tokenizer(
    [
        prompt.format(
            article_data["Titular"], # titular
            article_data["Copete"], # summary
            article_data["Cuerpo"],# body
    ""
    )], return_tensors = "pt").to("cuda")

    outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
    return tokenizer.batch_decode(outputs)

In [22]:
for i in range(4):
    print(sensationalist_dataset.iloc[i]["Amarillismo"])
    #print(inference(sensationalist_dataset.iloc[i])[0][-30:])

Amarillista
Amarillista
Amarillista
Amarillista


In [11]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    prompt.format(
        "¿Un parque Jurásico de la vida real?: podrían clonar dinosaurios después de un increíble descubrimiento de fósiles", # titular
        "Los expertos lograron descubrir células de ADN sanas en un dinosaurio fósil y se podría experimentar un escenario de parque jurásico de la vida real si los científicos pueden clonar al animal. Los detalles", # summary
        '''Los científicos podrían estar a punto de experimentar un gran avance al estilo Jurassic Park después de descubrir ADN saludable en un fósil de dinosaurio en China.El fósil desenterrado podría permitir la clonación de la criatura prehistórica y permitir experimentar escenas icónicas de la película de 1993 en la vida real. 
Los expertos creen que su fascinante descubrimiento podría contener el primer código genético prehistórico que aún no se descubrió.El escenario onírico podría convertirse en realidad si los profesionales pueden extraer cuidadosamente la información del fósil perfectamente conservado.

El fósil, que es una pieza de cartílago del muslo de un Caudipteryx se mantuvo en la ceniza volcánica, informó The Sun.

La aterradora criatura, que se parecía a los populares velociraptores de la película, caminó sobre la tierra hace unos 125 millones de años. La especie era conocida por su inteligencia y podía alcanzar velocidades de 24 millas por hora, lo que significa que podían cubrir una gran cantidad de terreno.

Fue descubierto por un equipo de la Academia de Ciencias de China que dijo que estaba en ""exquisitas"" condiciones. Los expertos dijeron que pudieron rastrear algunas células sanas entre otras que estaban enfermas al morir, informó un estudio.

Las células de cartílago bien conservadas se mineralizaron a través de un proceso llamado silicificación y no podrían considerarse ""roca"", ya que contienen restos de moléculas orgánicas. La profesora Alida Bailleul comentó: ""Obviamente estamos interesados en los núcleos de células fosilizadas porque aquí es donde estará la mayor parte del ADN, si se conserva el ADN. Tenemos datos preliminares buenos y emocionantes. Necesitamos averiguar exactamente qué son esas moléculas orgánicas, pero espero que podamos reconstruir una secuencia de ADN''',# body
""
)], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


['<|begin_of_text|> \nBelow you will find the title, summary, and body of a news article. \nYour task is to analyze these components and classify whether the article is sensationalist or not.\n\n### Article information:\n    Title: ¿Un parque Jurásico de la vida real?: podrían clonar dinosaurios después de un increíble descubrimiento de fósiles\n    Summary: Los expertos lograron descubrir células de ADN sanas en un dinosaurio fósil y se podría experimentar un escenario de parque jurásico de la vida real si los científicos pueden clonar al animal. Los detalles\n    Body: Los científicos podrían estar a punto de experimentar un gran avance al estilo Jurassic Park después de descubrir ADN saludable en un fósil de dinosaurio en China.El fósil desenterrado podría permitir la clonación de la criatura prehistórica y permitir experimentar escenas icónicas de la película de 1993 en la vida real. \nLos expertos creen que su fascinante descubrimiento podría contener el primer código genético pre

In [12]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    prompt.format(
        "SETI captura senales de radio anomalas", # titular
        "Posible nueva pista en la busqueda de vida extraterrestre", # summary
        '''El SETI Institute ha capturado senales de radio que no coinciden con patrones conocidos de fuentes naturales, lo que ha despertado interes sobre su posible origen extraterrestre.''',# body
""
)], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


['<|begin_of_text|> \nBelow you will find the title, summary, and body of a news article. \nYour task is to analyze these components and classify whether the article is sensationalist or not.\n\n### Article information:\n    Title: SETI captura senales de radio anomalas\n    Summary: Posible nueva pista en la busqueda de vida extraterrestre\n    Body: El SETI Institute ha capturado senales de radio que no coinciden con patrones conocidos de fuentes naturales, lo que ha despertado interes sobre su posible origen extraterrestre.\n\n### is this article sensationalist?:\n1<|end_of_text|>']

In [13]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    prompt.format(
        "Cambridge descubre bacteria que come plastico", # titular
        "Podria ayudar a resolver la crisis de contaminacion por plastico", # summary
        '''Cientificos de la University of Cambridge han descubierto una bacteria que puede descomponer rapidamente los plasticos comunes, ofreciendo una posible solucion a la contaminacion global''',# body
""
)], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


['<|begin_of_text|> \nBelow you will find the title, summary, and body of a news article. \nYour task is to analyze these components and classify whether the article is sensationalist or not.\n\n### Article information:\n    Title: Cambridge descubre bacteria que come plastico\n    Summary: Podria ayudar a resolver la crisis de contaminacion por plastico\n    Body: Cientificos de la University of Cambridge han descubierto una bacteria que puede descomponer rapidamente los plasticos comunes, ofreciendo una posible solucion a la contaminacion global\n\n### is this article sensationalist?:\n0<|end_of_text|>']

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Continue the fibonnaci sequence.", # instruction
        "1, 1, 2, 3, 5, 8", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Continue the fibonnaci sequence.

### Input:
1, 1, 2, 3, 5, 8

### Response:
13, 21, 34, 55, 89, 144<|end_of_text|>


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!

inputs = tokenizer(
[
    alpaca_prompt.format(
        "What is a famous tall tower in Paris?", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


["<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nWhat is a famous tall tower in Paris?\n\n### Input:\n\n\n### Response:\nThe Eiffel Tower is a famous landmark in Paris, France. It is a wrought iron tower that was built in 1889 for the World's Fair. Standing at 324 meters tall, it is the tallest building in Paris and one of the most recognizable landmarks in the world.<|end_of_text|>"]

You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [ ]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in `llama.cpp` or a UI based system like `GPT4All`. You can install GPT4All by going [here](https://gpt4all.io/index.html).

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/u54VK8m8tk) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Zephyr DPO 2x faster [free Colab](https://colab.research.google.com/drive/15vttTpzzVXv_tJwEk-hIcQ0S9FcEWvwP?usp=sharing)
2. Llama 7b 2x faster [free Colab](https://colab.research.google.com/drive/1lBzz5KeZJKXjvivbYvmGarix9Ao6Wxe5?usp=sharing)
3. TinyLlama 4x faster full Alpaca 52K in 1 hour [free Colab](https://colab.research.google.com/drive/1AZghoNBQaMDgWJpi4RbffGM1h6raLUj9?usp=sharing)
4. CodeLlama 34b 2x faster [A100 on Colab](https://colab.research.google.com/drive/1y7A0AxE3y8gdj4AVkl2aZX47Xu3P1wJT?usp=sharing)
5. Mistral 7b [free Kaggle version](https://www.kaggle.com/code/danielhanchen/kaggle-mistral-7b-unsloth-notebook)
6. We also did a [blog](https://huggingface.co/blog/unsloth-trl) with 🤗 HuggingFace, and we're in the TRL [docs](https://huggingface.co/docs/trl/main/en/sft_trainer#accelerate-fine-tuning-2x-using-unsloth)!
7. `ChatML` for ShareGPT datasets, [conversational notebook](https://colab.research.google.com/drive/1Aau3lgPzeZKQ-98h69CCu1UJcvIBLmy2?usp=sharing)
8. Text completions like novel writing [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing)
9. [**NEW**] We make Phi-3 Medium / Mini **2x faster**! See our [Phi-3 Medium notebook](https://colab.research.google.com/drive/1hhdhBa1j_hsymiW9m-WzxQtgqTH_NHqi?usp=sharing)

<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/u54VK8m8tk"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://ko-fi.com/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Kofi button.png" width="145"></a></a> Support our work if you can! Thanks!
</div>